# Train and evaluate a PC

In [1]:
import random
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cpu")  # The device to use, e.g., "cpu", "cuda", "cuda:1"

%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

Set the random seeds.

In [3]:
random.seed(4)
np.random.seed(4)
torch.manual_seed(4)
# if 'cuda' in device.type:
#     torch.cuda.manual_seed(42)

## Load MNIST Dataset

Load the training and test splits of MNIST, and preprocess them by flattening the tensor images.

In [ ]:
from torchvision import transforms, datasets
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: (255 * x.view(-1)).long())
])
data_train = datasets.MNIST('datasets', train=True, download=True, transform=transform)
data_test = datasets.MNIST('datasets', train=False, download=True, transform=transform)
num_variables = data_train[0][0].shape[0]
height, width = 28, 28
print(f"Number of variables: {num_variables}")

In [ ]:
plt.matshow(data_train[0][0].reshape(28, 28), cmap='gray')
plt.title(f"Class: {data_train[0][1]}")
plt.show()

## Instantiating the region graph

Initialize a _Quad Graph_ region graph.

In [76]:
from cirkit.region_graph.quad_tree import QuadTree
# region_graph = QuadTree(width, height, struct_decomp=False)
# region_graph = RandomBinaryTree(num_vars=128, depth=6, num_repetitions=1)
region_graph = FullyFactorized(num_vars=8)

In [ ]:
region_graph

In [ ]:
region_graph._nodes

Others available region graphs are _Poon Domingos_ and _QuadTree_, whose imports are showed below.

In [6]:
from cirkit.region_graph.poon_domingos import PoonDomingos
from cirkit.region_graph.random_binary_tree import RandomBinaryTree
from cirkit.region_graph.fully_factorized import FullyFactorized

## Choosing the layers

Now we have to choose both the input and inner layers of our circuit. As input layer we select the _CategoricalLayer_ with 256 categories (the number of pixel values). For the inner layer instead, we choose the _uncollapsed CP_ layer with rank 1.

In [77]:
from cirkit.layers.input.exp_family import CategoricalLayer
from cirkit.layers.sum_product import CPLayer
from cirkit.layers.sum_product.cp_w_bias import CPLayerWithBias
from cirkit.layers.input.rbf_network_kernel import RBFNetworkKernelLayer

efamily_cls = RBFNetworkKernelLayer
efamily_kwargs = {}
layer_cls = CPLayerWithBias
layer_kwargs = {'rank': 1}

## Building the tensorized PC

We can now build our tensorized PC by specifying the region graph and layers we chose previously. In addition, we can scale the architecture by increasing the number of input and inner units. We can also have circuits with multiple output units by choosing _num_classes > 1_. However, in this notebook we only estimate the distribution of the images and marginalize out the class variable.

To ensure weights are non-negative we reparametrize them via exponentiation. Several reparametrization functions are available.

In [78]:
from cirkit.reparams.leaf import ReparamExp, ReparamLogSoftmax, ReparamSoftmax, ReparamIdentity
from cirkit.models.tensorized_circuit import TensorizedPC
pc_rbfn = TensorizedPC.from_region_graph(
    region_graph,
    num_inner_units=10,
    num_input_units=10,
    efamily_cls=efamily_cls,
    efamily_kwargs=efamily_kwargs,
    layer_cls=layer_cls,
    layer_kwargs=layer_kwargs,
    num_classes=1,
    reparam=ReparamLogSoftmax # ReparamExp ReparamIdentity # 
)
pc_rbfn.to(device)
print(pc_rbfn)

TensorizedPC(
  (input_layer): RBFNetworkKernelLayer(
    (params_sigma): ReparamExp()
    (params_mu): ReparamIdentity()
  )
  (scope_layer): ScopeLayer()
  (inner_layers): ModuleList(
    (0): CollapsedCPLayer(
      (params_in): ReparamLogSoftmax()
      (params_bias): ReparamLogSoftmax()
    )
  )
)


In [79]:
for param in pc_rbfn.parameters(): 
    print (param.shape)

torch.Size([8, 10])
torch.Size([8, 10])
torch.Size([1, 8, 10, 1])
torch.Size([1, 1])


In [96]:
from cirkit.models.gp import CircuitGP, initial_values

In [4]:
import torch.nn.functional as F

from uci_datasets import Dataset

from ignite.engine import Events, Engine
from ignite.metrics import Average, Loss
from ignite.contrib.handlers import ProgressBar

import gpytorch
from gpytorch.mlls import VariationalELBO
from gpytorch.likelihoods import GaussianLikelihood

import pandas as pd
import numpy as np


In [20]:
data = Dataset("protein")
x_train, y_train, x_test, y_test = data.get_split(split=2)

protein dataset, N=45730, d=9


In [6]:
x_train.shape, x_test.shape

((41157, 9), (4573, 9))

In [21]:
x_train_real = x_train[:36584] #32000 # 2053   36584    36584     39063   13281    2672   # RE-RUN # 13279   # 1279   4701  824
y_train_real = y_train[:36584]
y_train_real = y_train_real.squeeze()
x_val = x_train[36584:]
y_val = y_train[36584:]
y_val = y_val.squeeze()
y_test = y_test.squeeze()

In [22]:
mean = x_train_real.mean(axis=0)
std = x_train_real.std(axis=0)

x_train_real_normalized = (x_train_real - mean) / std
x_val_normalized = (x_val - mean) / std
x_test_normalized = (x_test - mean) / std

# x_train_real_normalized = x_train_real
# x_val_normalized = x_val
# x_test_normalized = x_test


In [338]:
x_train_real.dtype

dtype('float64')

In [23]:
np.random.seed(24)
torch.manual_seed(24) ####################### CHANGE

batch_size = 32 # 64

ds_train = torch.utils.data.TensorDataset(torch.from_numpy(x_train_real_normalized).float(), torch.from_numpy(y_train_real).float())
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, drop_last=True) # suffle 

ds_val = torch.utils.data.TensorDataset(torch.from_numpy(x_val_normalized).float(), torch.from_numpy(y_val).float())
dl_val = torch.utils.data.DataLoader(ds_val, batch_size=512, shuffle=False)

ds_test = torch.utils.data.TensorDataset(torch.from_numpy(x_test_normalized).float(), torch.from_numpy(y_test).float())
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=512, shuffle=False)

In [24]:
from cirkit.region_graph.poon_domingos import PoonDomingos
from cirkit.region_graph.random_binary_tree import RandomBinaryTree
from cirkit.region_graph.fully_factorized import FullyFactorized
from cirkit.region_graph.quad_tree import QuadTree
# region_graph = QuadTree(width, height, struct_decomp=False)
region_graph = RandomBinaryTree(num_vars=9, depth=3, num_repetitions=10)
# region_graph = FullyFactorized(num_vars=8)


from cirkit.layers.input.exp_family import CategoricalLayer
from cirkit.layers.sum_product import CPLayer
from cirkit.layers.sum_product.cp_w_bias import CPLayerWithBias
from cirkit.layers.input.rbf_network_kernel import RBFNetworkKernelLayer

efamily_cls = RBFNetworkKernelLayer
efamily_kwargs = {}
layer_cls = CPLayerWithBias
layer_kwargs = {'rank': 1}

num_units = 128 # 356


from cirkit.reparams.leaf import ReparamExp, ReparamLogSoftmax, ReparamSoftmax, ReparamIdentity
from cirkit.models.tensorized_circuit import TensorizedPC
pc_rbfn = TensorizedPC.from_region_graph(
    region_graph,
    num_inner_units=num_units,
    num_input_units=num_units,
    efamily_cls=efamily_cls,
    efamily_kwargs=efamily_kwargs,
    layer_cls=layer_cls,
    layer_kwargs=layer_kwargs,
    num_classes=1,
    reparam=ReparamIdentity # ReparamExp  # ReparamLogSoftmax
)
pc_rbfn.to(device)
print(pc_rbfn)


TensorizedPC(
  (input_layer): RBFNetworkKernelLayer(
    (params_sigma): ReparamExp()
    (params_mu): ReparamIdentity()
    (params_weight): ReparamIdentity()
  )
  (scope_layer): ScopeLayer()
  (inner_layers): ModuleList(
    (0-2): 3 x CollapsedCPLayer(
      (params_in): ReparamIdentity()
    )
    (3): SumLayer(
      (params): ReparamIdentity()
    )
  )
)


In [25]:
for param in pc_rbfn.parameters(): 
    print (param.shape)

torch.Size([9, 128])
torch.Size([9, 128])
torch.Size([9, 128, 128])
torch.Size([40, 128, 128])
torch.Size([20, 128, 128])
torch.Size([10, 128, 1])
torch.Size([1, 10, 1])


In [26]:
total_params = sum(p.numel() for p in pc_rbfn.parameters() if p.requires_grad)
total_params

1134090

In [408]:
import torch.nn as nn

class IdentityMapping(nn.Module):
    def __init__(self):
        super(IdentityMapping, self).__init__()
    
    def forward(self, x):
        return x
    
feature_extractor = IdentityMapping()

initial_centers, initial_lengthscale = initial_values(
    ds_train, feature_extractor, n_inducing_points=num_units
)
initial_centers.transpose(0, 1), initial_lengthscale

pc_rbfn.input_layer.params_sigma.param = torch.nn.Parameter(
            torch.log(initial_lengthscale * torch.ones_like(pc_rbfn.input_layer.params_sigma.param) ))

pc_rbfn.input_layer.params_mu.param = torch.nn.Parameter(initial_centers.transpose(0, 1))

f_X_samples torch.Size([13281, 17])


/Users/irwinchay/opt/anaconda3/envs/cirkit1/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


initial_lengthscale tensor(5.6649)


In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(24)
np.random.seed(24)

# Parameters
batch_size = 32
epochs = 50
learning_rate = 1e-3

# Assume pc_rbfn is your model, already defined and initialized elsewhere

# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(pc_rbfn.parameters(), lr=learning_rate)


nan_counter = 0
exit_loops = False

# Training and Validation Loop
for epoch in range(epochs):
    # Training phase
    pc_rbfn.train()
    train_loss = 0.0
    
    for inputs, targets in dl_train:
        optimizer.zero_grad()
        outputs = pc_rbfn(inputs)
        outputs = outputs.squeeze(1)  # Ensure outputs match the target's shape
        
        # if(torch.isnan(outputs).any() == False):
        #     print("no NAN")

        if(torch.isnan(outputs).any()):
            print("outputs", outputs)
        
        if(torch.isnan(targets).any()):
            print("targets", targets)
        
        loss = criterion(outputs, targets)
        if(torch.isnan(loss).any()):
            print("loss", loss)
            nan_counter += 1
        if (nan_counter > 3):
            exit_loops = True
            break
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        
    if exit_loops:
        break
    train_loss /= len(dl_train.dataset)

    # Validation phase
    pc_rbfn.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in dl_val:
            outputs = pc_rbfn(inputs)
            outputs = outputs.squeeze(1)  # Ensure outputs match the target's shape
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
    val_loss /= len(dl_val.dataset)

    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")



Epoch 1: Train Loss: 0.4579, Val Loss: 0.3807
Epoch 2: Train Loss: 0.3632, Val Loss: 0.3516
Epoch 3: Train Loss: 0.3436, Val Loss: 0.3374
Epoch 4: Train Loss: 0.3336, Val Loss: 0.3628
Epoch 5: Train Loss: 0.3238, Val Loss: 0.3503
Epoch 6: Train Loss: 0.3124, Val Loss: 0.3145
Epoch 7: Train Loss: 0.3059, Val Loss: 0.3093
Epoch 8: Train Loss: 0.2971, Val Loss: 0.3055
Epoch 9: Train Loss: 0.2876, Val Loss: 0.3127
Epoch 10: Train Loss: 0.2764, Val Loss: 0.2857
Epoch 11: Train Loss: 0.2677, Val Loss: 0.2669
Epoch 12: Train Loss: 0.2574, Val Loss: 0.2632
Epoch 13: Train Loss: 0.2485, Val Loss: 0.2442
Epoch 14: Train Loss: 0.2418, Val Loss: 0.2566
Epoch 15: Train Loss: 0.2342, Val Loss: 0.2451
Epoch 16: Train Loss: 0.2275, Val Loss: 0.2427
Epoch 17: Train Loss: 0.2238, Val Loss: 0.2295
Epoch 18: Train Loss: 0.2182, Val Loss: 0.2547
Epoch 19: Train Loss: 0.2109, Val Loss: 0.2411
Epoch 20: Train Loss: 0.2060, Val Loss: 0.2321
Epoch 21: Train Loss: 0.1978, Val Loss: 0.2306
Epoch 22: Train Loss: 

In [29]:
# Testing
test_loss = 0.0
pc_rbfn.eval()  # Ensure model is in evaluation mode
with torch.no_grad():  # No gradients needed
    for inputs, targets in dl_test:
        outputs = pc_rbfn(inputs).squeeze(1)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
test_loss /= len(dl_test.dataset)
rmse = np.sqrt(test_loss)  # Calculate RMSE
print(f"Test RMSE: {rmse:.4f}")

Test RMSE: 0.4769


# MNIST

In [382]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision import datasets

# Set random seed for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Define the transformation to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Load MNIST dataset
train_val_dataset = datasets.MNIST('datasets', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('datasets', train=False, download=True, transform=transform)

# Split train dataset into train and validation
train_size = int(len(train_val_dataset) * 0.9)
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

# DataLoader
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)







In [383]:
for batch_idx, (data, target) in enumerate(train_loader):
    print(f"Batch index: {batch_idx}")
    print(f"Input tensor shape: {data.shape}")  # Shape will be [batch_size, channels, height, width] for images
    print(f"Target tensor shape: {target.shape}")  # Shape will be [batch_size] for the labels
    # Optionally break after the first batch to just see one example
    break

Batch index: 0
Input tensor shape: torch.Size([32, 784])
Target tensor shape: torch.Size([32])


In [384]:
from cirkit.region_graph.poon_domingos import PoonDomingos
from cirkit.region_graph.random_binary_tree import RandomBinaryTree
from cirkit.region_graph.fully_factorized import FullyFactorized
from cirkit.region_graph.quad_tree import QuadTree
region_graph = QuadTree(28, 28, struct_decomp=True)
# region_graph = RandomBinaryTree(num_vars=8, depth=3, num_repetitions=1)
# region_graph = FullyFactorized(num_vars=8)


from cirkit.layers.input.exp_family import CategoricalLayer
from cirkit.layers.sum_product import CPLayer
from cirkit.layers.sum_product.cp_w_bias import CPLayerWithBias
from cirkit.layers.input.rbf_network_kernel import RBFNetworkKernelLayer

efamily_cls = RBFNetworkKernelLayer
efamily_kwargs = {}
layer_cls = CPLayerWithBias
layer_kwargs = {'rank': 1}

num_units = 32 # 356


from cirkit.reparams.leaf import ReparamExp, ReparamLogSoftmax, ReparamSoftmax, ReparamIdentity
from cirkit.models.tensorized_circuit import TensorizedPC
pc_rbfn = TensorizedPC.from_region_graph(
    region_graph,
    num_inner_units=num_units,
    num_input_units=num_units,
    efamily_cls=efamily_cls,
    efamily_kwargs=efamily_kwargs,
    layer_cls=layer_cls,
    layer_kwargs=layer_kwargs,
    num_classes=10,
    reparam=ReparamIdentity # ReparamExp  # ReparamLogSoftmax
)
pc_rbfn.to(device)
print(pc_rbfn)


TensorizedPC(
  (input_layer): RBFNetworkKernelLayer(
    (params_sigma): ReparamExp()
    (params_mu): ReparamIdentity()
    (params_weight): ReparamIdentity()
  )
  (scope_layer): ScopeLayer()
  (inner_layers): ModuleList(
    (0-9): 10 x CollapsedCPLayer(
      (params_in): ReparamIdentity()
      (params_bias): ReparamIdentity()
    )
  )
)


In [357]:
for param in pc_rbfn.parameters():
    print(param.shape)

torch.Size([784, 32])
torch.Size([784, 32])
torch.Size([784, 32, 32])
torch.Size([288, 32, 32])
torch.Size([288, 32])
torch.Size([240, 32, 32])
torch.Size([240, 32])
torch.Size([128, 32, 32])
torch.Size([128, 32])
torch.Size([64, 32, 32])
torch.Size([64, 32])
torch.Size([32, 32, 32])
torch.Size([32, 32])
torch.Size([16, 32, 32])
torch.Size([16, 32])
torch.Size([8, 32, 32])
torch.Size([8, 32])
torch.Size([4, 32, 32])
torch.Size([4, 32])
torch.Size([2, 32, 32])
torch.Size([2, 32])
torch.Size([1, 32, 10])
torch.Size([1, 10])


In [386]:
# Instantiate the model, loss function, and optimizer
model = pc_rbfn
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training and validation loop
epochs = 10
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(val_loader)

    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Add your test loop here if needed, following the same pattern as validation
# Remember to evaluate the model's performance on the test set after training

Epoch 1, Train Loss: nan, Val Loss: nan


KeyboardInterrupt: 